https://towardsdatascience.com/symspell-vs-bk-tree-100x-faster-fuzzy-string-search-spell-checking-c4f10d80a078#:~:text=The%20Symmetric%20Delete%20spelling%20correction%20algorithm%20reduces%20the%20complexity%20of,%2B%20inserts)%20and%20language%20independent.

# Prepare Dataset

In [ ]:
! pip -q install datasets transformers symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
import transformers
from datasets import DatasetDict, load_dataset, ClassLabel, Dataset
import random
import pandas as pd
from IPython.display import display, HTML
from pprint import pprint
from tqdm import tqdm

import time

import re
import string
from collections import Counter
import numpy as np

print(transformers.__version__)

4.25.1


In [ ]:
## fdfdshf_AsmDXMNqSSPMWPpvStkqoMwJYTFslhSQsWho

! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    return df



### local csv

In [ ]:
my_codemixed_source = r'/content/drive/MyDrive/Homoglyphed code-mixed/Datsets/source/my_code_mixed_11k.csv'
IIITH_source = r'/content/drive/MyDrive/Homoglyphed code-mixed/Datsets/source/IIITH_Codemixed.csv'

datasets = load_dataset("csv", data_files=[my_codemixed_source,IIITH_source])

datasets = datasets.rename_column("content", "text")
datasets = datasets.remove_columns(['Unnamed: 0', 'label'])
datasets = datasets.filter(lambda example: len(example["text"].split())>3).shuffle(seed=42)
datasets['test'] = datasets['train']

# datasets = DatasetDict({
#     # 'train': Dataset.from_list(datasets['train']['content']),
#     'test': datasets['train'],
#     # 'valid': Dataset.from_dict(datasets['valid'][:test_n])
#     })

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0cd5a21bec5308d3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

### reduce samples for testing locally

In [ ]:
# test_n = 8_000_000
# datasets = DatasetDict({
#     'train': Dataset.from_dict(datasets['train'][:test_n]),
#     'test': Dataset.from_dict(datasets['test'][:test_n]),
#     'valid': Dataset.from_dict(datasets['valid'][:test_n])
#     })

In [ ]:
test_n = 5_000_000
datasets = DatasetDict({
    'train': Dataset.from_dict(datasets['train'][:test_n]),
    })

### Check dataset

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 13531
    })
    test: Dataset({
        features: ['text'],
        num_rows: 13531
    })
})

In [ ]:
random_tokens = show_random_elements(datasets["train"])

,text
0,Super duper hit.rahegi bhai
1,PM MODI JI BAHUT UDARWADI VIKAS PURUS HAIN
2,Tu bolke bhi kya aukhad lega pehle carry pe roast kiya toh anjam dekh kr samjh ni ayi kya or papa h tera or papa ko bolne ki aukaat ni h tumhari ohhhhhhhhhhhhhhhhhhhhhhhhhhhhh
3,Wah Sirji esi situation bhi aap ne kafi acchi bat kah di what a great Thaught thumbs_up OK_hand folded_hands
4,Tukde gang ki pawn
5,salman sir plz 5:00 pm kar dou plz 4:00 pm mein nai baat kar saku gi aapse plz sir plz
6,Tumhare baap main dumm hai to Arakshan hatake dikhao phir batat hu Jay Bhim arakshanjaruri
7,Wahh baba ramdev g ab Kya kahoge clapping_hands BabaRamdev SayNoToPatanjali ArrestRamdev
8,Wah Re ! fire fire
9,Ye kya jawab denge Mam


# Candidate word recommender by Symspell

https://github.com/mammothb/symspellpy

## new

In [ ]:
words = []
for line in tqdm(datasets["train"]):
  words += re.findall(r'\w+', line['text'].lower())

# vocabs = set(words)
word_counts = Counter(words)
# total_words = float(sum(word_counts.values()))
words_counts = dict(word_counts)


100%|██████████| 13531/13531 [00:01<00:00, 10526.63it/s]


In [ ]:
pd.DataFrame([[i for i in word_counts.keys()], [j for j in word_counts.values()]]).T.to_csv(r'/content/drive/MyDrive/Homoglyphed code-mixed/Spelling_corrector/frequency_dict/local_13k.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell()
corpus_path = r'/content/pandas.txt'
sym_spell.load_dictionary(corpus_path,0,1)

sym_spell._max_dictionary_edit_distance=10

print(sym_spell.words)
print(sym_spell._max_dictionary_edit_distance)

{'aap': 2544, 'ye': 5433, 'btaie': 3, 'ki': 6429, 'lucknow': 24, 'kyu': 423, 'nhi': 1803, 'ate': 18, 'hai': 13878, 'woh': 417, 'chali': 57, 'dekho': 249, 'pyar': 1644, 'gali': 60, 'ussey': 6, 'rokna': 3, 'koi': 1803, 'fatehsagar': 9, 'smiling_face_with_heart': 1149, 'eyes': 1245, 'kya': 9921, 'mujhe': 705, 'miss': 174, 'kar': 2115, 'rahe': 1176, 'ho': 5733, 'picture': 111, 'by': 480, 'udaipur': 72, 'udaipurdiaries': 12, 'udaipurdairies': 3, 'udaipurcity': 6, 'udaipurlove': 6, 'udaipurtimes': 6, 'udaipurite': 9, 'udaipurlakecity': 3, 'udaipurblog': 9, 'bhai': 4800, 'aapki': 216, 'movie': 708, 'chhappar': 3, 'far': 6, 'dega': 96, 'allah': 276, 'kare': 450, 'app': 303, 'film': 495, 'bajrangi': 591, 'bhaijaan': 960, 'south': 15, 'therator': 3, 'mey': 30, 'reliz': 3, 'jaye': 309, 'bharat': 309, 'mata': 63, 'jai': 372, 'wo': 1194, 'jahil': 9, 'h': 3141, 'na': 2043, 'hi': 3177, 'unne': 3, 'science': 21, 'pdi': 6, 'aur': 2403, 'quran': 18, 'me': 4926, 'saaf': 33, 'likha': 84, 'jo': 1671, 'apne

In [ ]:
suggestions = sym_spell.lookup('ldxgbbta', Verbosity.CLOSEST, max_edit_distance=4)
for i in suggestions:
  print(i)

lagta, 4, 240
lgta, 4, 21
dabba, 4, 9
dibba, 4, 6
lgata, 4, 6
dhbba, 4, 3
